In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# **Lab 3:** Vertex AI Model Deployment
This lab deploys our trained BQML model to Vertex AI. We will then submit our inference requests for prediction in real time!

In [ ]:
! pip install --quiet --upgrade google-cloud-aiplatform 

In [ ]:
project_id   = ""
team_name    = "" 
location     = "us" #This is currently necessary
region       = "us-central1"

dataset_name = "datathon_ds_{}".format(team_name)
bucket_name  = "gs://{}_{}".format(project_id,dataset_name)

In [ ]:
from typing import Dict, List, Union
from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
from google.cloud import bigquery
from google.cloud.bigquery import Client, QueryJobConfig
import json

client = bigquery.Client(project=project_id)

In [ ]:
! gcloud config set project $project_id

In [ ]:
! gsutil mb -l $region $bucket_name

Enable the **Vertex AI API**: https://console.cloud.google.com/marketplace/product/google/aiplatform.googleapis.com

## Deploying your BQML model to Vertex

**Step 1**: Export BQML model to GCS bucket

To export your trained baseline BQML model to the previously created bucket by follow the steps provided [[here]](https://cloud.google.com/bigquery/docs/exporting-models#export)

**Step 2** Import model to vertex model registry

We will now import the model to vertex ai model registry by clicking on the ```import``` button and following the steps mentioned [[here]](https://cloud.google.com/vertex-ai/docs/model-registry/import-model#import_a_model_using)

Under model settings select:
*   Model framework -> tensorflow
*   Model framework version -> 1.15
*   Accelerator type -> None
*   leave all other settings as default

**Step 3** Deploy model to vertex endpoint (~15 mins)

We will now deploy our model to an endpoint following the steps provided [[here]](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions#deploy_a_model_to_an_endpoint)

Under model settings select:
*   machine type -> n1-standard-2
*   leave all other settings as default
*   skip model monitoring section

Note that the model deployment will take several minutes (~15 mins)




## Run inference on Deployed Model in real time

To run your first real time prediction:
*   Select your model endpoint under deployments. 
*   Select the ```DEPLOY & TEST``` sub-menu.
*   Under the ```Test your model``` section paste the payload provided below


```
{"instances": [{"country":"India","operating_system":"ANDROID","language":"en-us","cnt_user_engagement": 72,"cnt_level_start_quickplay": 0,"cnt_level_end_quickplay": 6,"cnt_level_complete_quickplay": 3,"cnt_level_reset_quickplay": 1,"cnt_post_score": 9,"cnt_spend_virtual_currency": 0,"cnt_ad_reward": 0,"cnt_challenge_a_friend": 0,"cnt_completed_5_levels": 1,"cnt_use_extra_steps": 0,"user_first_engagement": 1533434460293005}
]}
```

Congrats!!!! You have successfully recived your first prediction

## Sending larger requests to the endpoint

In [ ]:
query = f"""SELECT * FROM `{dataset_name}.cc_eval_dataset`"""
job = client.query(query)
df = job.to_dataframe()

In [ ]:
df = df.drop(['user_pseudo_id', 'churned'], axis=1)


In [ ]:
df.head(2)

In [ ]:
df = df.dropna()

In [ ]:
# format the dataframe for the endpoint
payload = json.loads(df.to_json(orient="records"))

In [ ]:
def predict_custom_trained_model_sample(
    project: str,
    endpoint_id: str,
    instances: Union[Dict, List[Dict]],
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    """
    `instances` can be either single instance of type dict or a list
    of instances.
    """
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances = instances if type(instances) == list else [instances]
    instances = [
        json_format.ParseDict(instance_dict, Value()) for instance_dict in instances
    ]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    # The predictions are a google.protobuf.Value representation of the model's predictions.
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))

### Submit multiple prediction requests to model endpoint

In [ ]:
predict_custom_trained_model_sample(
    project="<your-project-id>",
    endpoint_id="your-vertex-endpoint-id",
    location="us-central1",
    instances=payload
)

## [Optional] Deploy a second version to the same endpoint

The next exercises are optional. You can continue to iterate on your BQML models for improving the ROC metric. Once you are comfortable with your models, feel free to explore the advanced features of vertex endpoints in the below sections.

Your second model was able to increase the accuracy of the ROC score. However, you probably don't want to update your application to point to a new endpoint URL, and you don't want to create sudden change in your application. You can add the new model to the same endpoint, serving a small percentage of traffic, and gradually increase the traffic split for the new model until it is serving 100% of the traffic.

Now lets deploy a second version of our logistic model to the same endpoint and split the traffic 50% to each model. [[docs]](https://cloud.google.com/vertex-ai/docs/general/deployment#models-endpoint)

Note that deploying a new version to the same endpoint takes fewer minutes (~5 min)

You can now submit your payload and monitor the traffic to both models via the endpoints page :)

## [Optional] Lets monitor for prediction skew for incoming requests in our endpoint

Select ```Edit settings``` tab for your model endpoint 

Under model monitoring:
*  select monitoring interval -> 1 hour
*  select monitoring data window -> 1 hour
*  select sampling rate -> 100%

Under monitoring objectives: 
*  select prediction drift detection
*  Set the alerts to the following below

```
{"cnt_user_engagement":0.01,"country":0.01,"cnt_spend_virtual_currency":0.01,"user_first_engagement":0.01,"language":0.01,"cnt_level_complete_quickplay":0.01,"cnt_challenge_a_friend":0.01,"cnt_use_extra_steps":0.01,"cnt_level_start_quickplay":0.01,"cnt_ad_reward":0.01,"cnt_level_reset_quickplay":0.01,"cnt_level_end_quickplay":0.01,"operating_system":0.01,"cnt_completed_5_levels":0.01,"cnt_post_score":0.01}
```

In [ ]:
# create a seperate dataframe for data skew
df_data_skew = df.copy(deep=True)

In [ ]:
from random import randrange

# add skew to the integer fields of our evaluation dataset 
df_data_skew['cnt_spend_virtual_currency'] = [ randrange(10000,100000)  for k in df_data_skew.index]
df_data_skew['cnt_user_engagement'] = [ randrange(2000,5000)  for k in df_data_skew.index]
df_data_skew['cnt_challenge_a_friend'] = [ randrange(10,20)  for k in df_data_skew.index]

In [ ]:
df_data_skew.describe()

In [ ]:
skew_payload = json.loads(df_data_skew.to_json(orient="records"))

In [ ]:
predict_custom_trained_model_sample(
    project="<your-project-id>",
    endpoint_id="your-vertex-endpoint-id",
    location="us-central1",
    instances=skew_payload
)